In [1]:
import os

os.chdir("../")
%pwd

'd:\\anjali_proj'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig():
    root_dir:Path
    source_bucket:str
    filename:str
    local_data_file:Path
    unzip_dir:Path

In [3]:
from currencyClassifier.constants import *
from currencyClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager():
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_bucket=config.source_bucket,
            filename=config.filename,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os
import zipfile
from currencyClassifier import logger
from currencyClassifier.utils.common import get_size
import boto3

In [6]:
class DataIngestion():
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def get_file(self):
        if not os.path.exists(self.config.local_data_file):
            s3 = boto3.client('s3')
            s3 = boto3.resource(
                service_name='s3',
                region_name='us-east-1',
                aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID'),
                aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
            )
            obj = s3.Bucket(self.config.source_bucket).Object(self.config.filename)
            filename = obj.key
            
            s3.Bucket(self.config.source_bucket).download_file(Key=self.config.filename, Filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded from S3 bucket!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.get_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-08-28 04:00:42,857: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 04:00:42,871: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 04:00:42,871: INFO: common: created directory at: artifacts]
[2024-08-28 04:00:42,871: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-28 04:00:42,916: INFO: credentials: Found credentials in environment variables.]
[2024-08-28 04:00:59,507: INFO: 1523282916: Currency_image.zip downloaded from S3 bucket!]
